# Dask Bag

Материалы: 
* Макрушин С.В. Лекция 12: Map-Reduce
* https://docs.dask.org/en/latest/bag.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Подсчитайте длину (в кол-ве символов) каждого предложения.

In [1]:
from nltk import sent_tokenize

text = open('./data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt',  encoding = 'windows-1251').read()
sents = sent_tokenize(text)
sents[:3]

['Спасибо, что скачали книгу в бесплатной электронной библиотеке BooksCafe.Net: http://bookscafe.net\n\nВсе книги автора: http://bookscafe.net/author/dostoevskiy_fedor-1096.html\n\nЭта же книга в других форматах: http://bookscafe.net/book/dostoevskiy_fedor-igrok-240117.html\n\nПриятного чтения!',
 'Федор Михайлович Достоевский\n\nИгрок\n\n(Из записок молодого человека)\n\n\n\n\nГлава I\n\nНаконец я возвратился из моей двухнедельной отлучки.',
 'Наши уже три дня как были в Рулетенбурге.']

In [2]:
list(map(len, sents))[:3]

[287, 133, 41]

2. Считайте файл `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt` и разбейте на предложения. Выведите предложения, длина которых не более 10 символов.

In [3]:
[sent for sent in sents if len(sent)<=10][:3]

['— ton).]', '—\xa0Плюнуть?', '«Как!']

In [4]:
list(filter(lambda sent: len(sent)<=10, sents))[:3]

['— ton).]', '—\xa0Плюнуть?', '«Как!']

3. На основе списка предложений из задачи 1-2 создайте `dask.bag`. Рассчитайте среднюю длину предложений в тексте.

In [5]:
import dask.bag as db

In [6]:
bag = db.from_sequence(sents)
bag.map(len).mean().compute()

72.89840944073885

4. На основе файла `addres_book.json` создайте `dask.bag`. Посчитайте количество мобильных и рабочих телефонов в наборе данных

In [7]:
import json
data = json.load(open("./data/addres-book.json"))
data[0]

{'name': 'Faina Lee',
 'email': 'faina@mail.ru',
 'birthday': '22.08.1994',
 'phones': [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}]}

In [8]:
bag = db.from_sequence(data)
bag

dask.bag<from_sequence, npartitions=8>

In [9]:
bag.map(lambda record: record["phones"]).compute()
bag.pluck("phones").compute()

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}],
 [{'phone': '232-19-56'}, {'phone': '+7 (916) 232-19-56'}],
 [{'phone': '+7 (916) 445-19-57'}],
 [{'phone': '232-19-58'}, {'phone': '+7 (916) 232-19-58'}],
 [{'phone': '+7 (916) 445-19-59'}],
 [{'phone': '232-19-50'}, {'phone': '+7 (916) 232-19-50'}],
 [{'phone': '111-19-51'}, {'phone': '+7 (916) 445-19-51'}]]

In [10]:
bag.pluck("phones").flatten().compute()

[{'phone': '232-19-55'},
 {'phone': '+7 (916) 232-19-55'},
 {'phone': '111-19-55'},
 {'phone': '232-19-56'},
 {'phone': '+7 (916) 232-19-56'},
 {'phone': '+7 (916) 445-19-57'},
 {'phone': '232-19-58'},
 {'phone': '+7 (916) 232-19-58'},
 {'phone': '+7 (916) 445-19-59'},
 {'phone': '232-19-50'},
 {'phone': '+7 (916) 232-19-50'},
 {'phone': '111-19-51'},
 {'phone': '+7 (916) 445-19-51'}]

In [11]:
def add_type(record):
    record["type"] = "mobile" if record["phone"].startswith("+7") else "work"
    return record

bag.pluck("phones").flatten().map(add_type).compute()

[{'phone': '232-19-55', 'type': 'work'},
 {'phone': '+7 (916) 232-19-55', 'type': 'mobile'},
 {'phone': '111-19-55', 'type': 'work'},
 {'phone': '232-19-56', 'type': 'work'},
 {'phone': '+7 (916) 232-19-56', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-57', 'type': 'mobile'},
 {'phone': '232-19-58', 'type': 'work'},
 {'phone': '+7 (916) 232-19-58', 'type': 'mobile'},
 {'phone': '+7 (916) 445-19-59', 'type': 'mobile'},
 {'phone': '232-19-50', 'type': 'work'},
 {'phone': '+7 (916) 232-19-50', 'type': 'mobile'},
 {'phone': '111-19-51', 'type': 'work'},
 {'phone': '+7 (916) 445-19-51', 'type': 'mobile'}]

In [12]:
bag.pluck("phones").flatten().map(add_type).pluck("type").frequencies().compute()

[('work', 6), ('mobile', 7)]

In [13]:
def binop(total, value):
    print(total, value)
    return total + 1

def combine(left, right):
    return left + right

In [14]:
bag.pluck("phones").flatten().map(add_type).foldby("type",binop, 0, combine).compute()

0 {'phone': '232-19-50', 'type': 'work'}
0 {'phone': '+7 (916) 232-19-50', 'type': 'mobile'}
0 {'phone': '111-19-51', 'type': 'work'}
0 {'phone': '+7 (916) 445-19-51', 'type': 'mobile'}
0 {'phone': '232-19-55', 'type': 'work'}
0 {'phone': '+7 (916) 232-19-55', 'type': 'mobile'}
0 {'phone': '111-19-55', 'type': 'work'}
0 {'phone': '232-19-56', 'type': 'work'}
0 {'phone': '+7 (916) 232-19-56', 'type': 'mobile'}
0 {'phone': '+7 (916) 445-19-57', 'type': 'mobile'}
0 {'phone': '232-19-58', 'type': 'work'}
0 {'phone': '+7 (916) 232-19-58', 'type': 'mobile'}
0 {'phone': '+7 (916) 445-19-59', 'type': 'mobile'}


[('work', 6), ('mobile', 7)]

## Лабораторная работа 12

1. В файлах архиве `reviews_full.zip` находятся файлы, содержащие информацию об отзывах к рецептам в формате JSON Lines. Отзывы разделены на файлы в зависимости от оценки (например, в файле `reviews_1.json` находятся отзывы с оценкой 1). Считайте файлы из этого архива в виде `dask.bag`. Преобразуйте текстовое содержимое файлов в объекты python (с помощью модуля `json`). Выведите на экран первые 5 элементов полученного `bag`.

In [15]:
import dask.bag as db
import json
import re
import pandas as pd

In [16]:
data_path = './data'
b = db.read_text(urlpath="./data/reviews_full/reviews_*.json").map(json.loads)
b.take(5)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': '2016-05-08',
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.'},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': '2006-06-14',
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten'},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': '1985-11-19',
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a perfect combo for us. The sausage gave it nice flavor No guestion , I will be making this often.'},
 {'u

2. Модифицируйте функцию разбора JSON таким образом, чтобы в каждый словарь c информацией об отзыве добавить ключ `rating`. Значение получите на основе названия файла (см. аргумент `include_path`), использовав для этого регулярное выражение.

In [17]:
import re
from datetime import datetime

In [18]:
def my_loader(data) -> dict:

    json_str, path = data
    file_number = re.findall('reviews_([0-9]).json', path)
    
    if file_number is None or len(file_number) != 1:
        raise ValueError("Не могу извлечь номер файла")
    
    json_obj = json.loads(json_str)
    json_obj["rating"] = int(file_number[0])
    json_obj["date"] = datetime.strptime(json_obj["date"], '%Y-%m-%d')

    return json_obj

data_path = './data'
b_bag = db.read_text(urlpath="./data/reviews_full/reviews_*.json", include_path=True).map(my_loader)
b_bag.take(50)

({'user_id': 452355,
  'recipe_id': 292657,
  'date': datetime.datetime(2016, 5, 8, 0, 0),
  'review': 'WOW!!! This is the best. I have never been able to make homemade enchiladas that taste like the Mexican restaurants. I made this last night for my family and they said they will never have enchiladas at the Mexican Restaurants again. Thanks for sharing.',
  'rating': 0},
 {'user_id': 329304,
  'recipe_id': 433404,
  'date': datetime.datetime(2006, 6, 14, 0, 0),
  'review': 'This was good but the dressing needed something and I found it to be a little too sweet, next time I will experiment with some garlic and herbs and reduce the sugar slightly, thanks for sharing kcdlong!...Kitten',
  'rating': 0},
 {'user_id': 227932,
  'recipe_id': 2008187,
  'date': datetime.datetime(1985, 11, 19, 0, 0),
  'review': 'Very good,it was a hit for my family. I used 6 cloves of garlic and had 1 lb beef and  Johnsonville sausage,1/2 lb hot and  1/2 lb honey garlic( which I wanted to use). That was a pe

3. Посчитайте количество отзывов в исходном датасете.

In [19]:
b_bag.count().compute()

9057540

4. Отфильтруйте `bag`, сохранив только отзывы, оставленные в 2014 и 2015 годах.

In [20]:
def years_filter(item):
    return item["date"].year in (2014, 2015)

In [21]:
c_bag = b_bag.filter(years_filter)
c_bag.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': datetime.datetime(2014, 10, 3, 0, 0),
  'review': 'Took this to a New Year&#039;s Eve Party. Everyone loved it! It&#039;s absolutely perfect, the flavor, the crunch, just delicious!',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': datetime.datetime(2015, 5, 8, 0, 0),
  'review': 'Simple and easy way to enjoy a slice of pizza any time!  Well-toasted bread is the key - really toast it!  I put a bit of pizza sauce underneath my cheese for a more pizza-like flavor.  I used sourdough bread & medium cheddar cheese.  Fast & fun!  Great idea!  Made for 1-2-3 Hits Tag Game.',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': datetime.datetime(2015, 6, 30, 0, 0),
  'review': 'Delish!  I wanted to make this spicy so I used hot enchilada sauce and jalapeno refried beans.  I forgot to buy the onions so I doctored up the beans with onion powder and granulated garlic.  Added the olives under the cheese 

5. Выполните препроцессинг отзывов:
    * привести строки к нижнему регистру
    * обрезать пробельные символы в начале и конце строки
    * удалите все символы, кроме английских букв и пробелов
    
Примените препроцессинг ко всем записям из `bag`, полученного в задании 4.

In [22]:
import re
pattern = re.compile('[^a-z ]+')

In [23]:
def my_preprocessing(item):
    current_str = item["review"].lower().strip()
    item["review"] = re.sub(pattern, '', current_str)
    return item

d_bag = c_bag.map(my_preprocessing)
d_bag.take(5)

({'user_id': 229850,
  'recipe_id': 1300038,
  'date': datetime.datetime(2014, 10, 3, 0, 0),
  'review': 'took this to a new years eve party everyone loved it its absolutely perfect the flavor the crunch just delicious',
  'rating': 0},
 {'user_id': 2706705,
  'recipe_id': 133747,
  'date': datetime.datetime(2015, 5, 8, 0, 0),
  'review': 'simple and easy way to enjoy a slice of pizza any time  welltoasted bread is the key  really toast it  i put a bit of pizza sauce underneath my cheese for a more pizzalike flavor  i used sourdough bread  medium cheddar cheese  fast  fun  great idea  made for  hits tag game',
  'rating': 0},
 {'user_id': 945545,
  'recipe_id': 898468,
  'date': datetime.datetime(2015, 6, 30, 0, 0),
  'review': 'delish  i wanted to make this spicy so i used hot enchilada sauce and jalapeno refried beans  i forgot to buy the onions so i doctored up the beans with onion powder and granulated garlic  added the olives under the cheese and baked uncovered for the  minutes  

6. Посчитайте количество отзывов в датасете, полученном в результате решения задачи 5. В случае ошибок прокомментируйте результат и исправьте функцию препроцессинга.

In [24]:
c_len = c_bag.count().compute()
d_len = d_bag.count().compute()
assert c_len == d_len
print(d_len)

735274


7. Посчитайте, как часто в наборе, полученном в задании 5, встречается та или иная оценка.

In [25]:
for item in d_bag.pluck(key="rating").frequencies().compute():
    print("Rating={}, count={}".format(*item))

Rating=0, count=42472
Rating=1, count=9246
Rating=2, count=9380
Rating=3, count=26532
Rating=4, count=119413
Rating=5, count=528231


8. Найдите среднее значение `rating` в наборе, полученном в задании 5.

In [26]:
d_bag.pluck(key="rating").mean().compute()

4.388036296673077

9. Используя метод `foldby`, подсчитать максимальную длину отзывов в зависимости от оценки `rating` в наборе, полученном в задании 5.

In [27]:
def combine(left, right):
    """Возврат максимального значения из всех сегментов"""
    return max(left, right)

def binop(previous, current):
    """Подсчет максимума внутри определенного сегмента"""
    return max(previous, len(current["review"]))

result = d_bag.foldby(key="rating", binop=binop, initial=0, combine=combine, combine_initial=0).compute()
for item in result:
    print("Rating={}, max review len={}".format(*item))

Rating=0, max review len=6548
Rating=1, max review len=2868
Rating=2, max review len=2834
Rating=3, max review len=3174
Rating=4, max review len=6548
Rating=5, max review len=5343
